In [1]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train=pd.read_csv('ACT11_competition_training.csv', dtype={"MOLECULE": object, "Act": float})

                  

Get truth values and reshape

In [3]:
y=train['Act'].values
y.shape

(6399,)

In [4]:
y=np.reshape(y,(-1,1))
y.shape

(6399, 1)

In [5]:
train=train.drop(['Act', 'MOLECULE'], axis=1)

In [6]:
x=train.values

In [7]:
####Provided by MERCK
def r_square(X, Y):
    """ r^2 value defined by the competition host, r^2 = 1 indicates 100% prediction accuracy
    """
    avx = np.mean(X)
    avy = np.mean(Y)
    sum1, sumx, sumy = 0, 0, 0
    for i in range(len(X)):
        sum1 += (X[i] - avx)*(Y[i] - avy)
        sumx += (X[i] - avx)*(X[i] - avx)
        sumy += (Y[i] - avy)*(Y[i] - avy)
    print(len(X), sum1, sumx, sumy)
    return sum1*sum1/(sumx*sumy)

In [8]:
# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer 1 with ReLu activation
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_1 = tf.nn.dropout(layer_1, keep_prob) 
    
    # Hidden layer 2 with ReLu activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_2 = tf.nn.dropout(layer_2, keep_prob) 
    
    # Output layer with ReLu activation
    out_layer = tf.add(tf.matmul(layer_2, weights['out']), biases['out'])
    return out_layer

In [9]:
n_hidden_1_array = [5,10,50,100]
n_hidden_2_array = [2,5,25,25]

In [10]:
for index_xx in range(10):
    print("-------------------- round "+str(index_xx)+"-----------------------")    
    
    for hide_i in range(len(n_hidden_1_array)):
    
        n_hidden_1 = n_hidden_1_array[hide_i]
        n_hidden_2 = n_hidden_2_array[hide_i]
        
        print("["+ str(n_hidden_1) +","+ str(n_hidden_2) +"]")
    
        seed = round(np.random.uniform(1, len(x)))
        X_train, X_dev, Y_train, Y_dev = train_test_split(x, y, train_size = 0.80, random_state = seed)
        X_val, X_test, Y_val, Y_test = train_test_split(X_dev, Y_dev, train_size = 0.50, random_state = seed)

        X_placeholder = tf.placeholder(tf.float32, (None, X_train.shape[1]))
        Y_placeholder = tf.placeholder(tf.float32, (None, Y_train.shape[1]))

        # define parameters
        features = np.shape(X_train)[1] # switch to X_train
        target_size = np.shape(X_train)[0]

        learning_rate = 0.001# switch to 0.05

        epochs = 900
        batch_size = 300

        batch_size_placeholder = tf.placeholder(tf.int64)

        # network parameters

    
        ds_train = tf.data.Dataset.from_tensor_slices(( X_placeholder, Y_placeholder )).shuffle(buffer_size=round(len(X_train) * 0.3)).batch(batch_size_placeholder)

        ds_test = tf.data.Dataset.from_tensor_slices(( X_placeholder, Y_placeholder )).batch( batch_size_placeholder )

        ds_iter = tf.data.Iterator.from_structure(ds_train.output_types, ds_train.output_shapes)

        next_x, next_y = ds_iter.get_next()

        train_init_op = ds_iter.make_initializer(ds_train)
        test_init_op = ds_iter.make_initializer(ds_test)
    
        # define placeholder for input vector X and target vector y
        keep_prob = tf.placeholder(tf.float32)

        # initialize weights and bias  
        weights = {'w1': tf.Variable(tf.truncated_normal([features, n_hidden_1], 0, 1, dtype=tf.float32)),
               'w2': tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2], 0, 1, dtype=tf.float32)),
              'out': tf.Variable(tf.truncated_normal([n_hidden_2, 1], 0, 1, dtype=tf.float32))}

        biases = {'b1': tf.Variable(tf.truncated_normal([n_hidden_1], 0, 1, dtype=tf.float32)),
              'b2': tf.Variable(tf.truncated_normal([n_hidden_2], 0, 1, dtype=tf.float32)),
             'out': tf.Variable(tf.truncated_normal([1], 0, 1, dtype=tf.float32))}
    
              
        # construct model
        y_pred = multilayer_perceptron(next_x, weights, biases)
    
        # define cost function(mean squred error) and optimizer(gradient descent)
        cost =  tf.losses.mean_squared_error(next_y, y_pred)
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    
        # initialize variables
        init_op = tf.global_variables_initializer()
    
        with tf.Session() as sess:
            sess.run(init_op)
            for epoch in range(epochs):

                sess.run(train_init_op, feed_dict={X_placeholder: X_train, Y_placeholder: Y_train, batch_size_placeholder: batch_size})
                count = 0

                while True:
                    try:
                        count += 1
                        _, c = sess.run((optimizer, cost), feed_dict={keep_prob: 0.75})
                    except tf.errors.OutOfRangeError:
                        break
#            print('Epoch:', (epoch + 1), 'cost =', c)
            sess.run(test_init_op, feed_dict={X_placeholder: X_test, Y_placeholder: Y_test, batch_size_placeholder: len(X_test)})
    
            results, test_cost = sess.run((y_pred, cost), feed_dict={keep_prob: 1.0})
    
            print(test_cost)
            print('R^2:', r_square(np.reshape(results, (len(results),)), Y_test))



-------------------- round 0-----------------------
[5,2]


/home/kevinyan/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


0.29380512
640 [-2.74623963e-22] 5.684341886080801e-13 [187.58375662]
R^2: [7.0729639e-34]
[10,5]
0.20375827
640 [36.03500548] 19.267813867443657 [183.09765551]
R^2: [0.368073]
[50,25]
0.2779622
640 [4.05028298] 0.7771119273660858 [181.19735876]
R^2: [0.11650251]
[100,25]
0.2878987
640 [2.30784277] 1.2166345775406455 [186.85983466]
R^2: [0.02342806]
-------------------- round 1-----------------------
[5,2]
0.28309512
640 [0.] 0.0 [180.98704932]
R^2: [nan]
[10,5]


/home/kevinyan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]


0.2356073
640 [22.48589475] 12.459809303189246 [180.41125984]
R^2: [0.22492893]
[50,25]
0.2529533
640 [1.0586242] 0.14425634509354102 [163.829181]
R^2: [0.04741955]
[100,25]
0.2781191
640 [4.94897866] 1.665969276743958 [185.05602144]
R^2: [0.07944397]
-------------------- round 2-----------------------
[5,2]
0.2625444
640 [-1.18452264e-21] 5.684341886080801e-13 [167.6393498]
R^2: [1.47241615e-32]
[10,5]
0.19830893
640 [31.11674527] 16.143369879043803 [172.95801685]
R^2: [0.34677952]
[50,25]
0.30122167
640 [0.00724685] 0.00046596978465096583 [192.79204184]
R^2: [0.00058459]
[100,25]
0.28804675
640 [-7.06412243e-22] 1.4210854715202004e-13 [183.98694387]
R^2: [1.908575e-32]
-------------------- round 3-----------------------
[5,2]
0.22755723
640 [34.16025813] 38.33631771560856 [175.58883931]
R^2: [0.17335445]
[10,5]
0.26511726
640 [5.50902288e-22] 5.684341886080801e-13 [169.54956139]
R^2: [3.14899732e-33]
[50,25]
0.29560152
640 [-0.22337666] 0.05033261683688295 [188.20271029]
R^2: [0.0052